
## Math GPT

* SFT
* GSM8K
* python 3.8
* use older version of TRL
* pip install trl==0.11.3



In [1]:

from accelerate import PartialState
import torch
from datasets import load_dataset
from tqdm.rich import tqdm
from transformers import AutoTokenizer, TrainingArguments
from trl import ModelConfig, SFTTrainer
tqdm.pandas()


In [2]:

## Reformat dataset prompts to  match the style that we want 
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        text = f"### Question: {example['question'][i]}\n ### Answer: {example['answer'][i]}"
        output_texts.append(text)
    return output_texts


In [3]:

## Training Args
training_args = TrainingArguments(
    output_dir="sft_gpt2",
    logging_strategy="epoch",
    num_train_epochs=3,
)


In [4]:

PartialState().process_index


0

In [5]:

device_string = PartialState().process_index

## Model & Tokenizer
model_config = ModelConfig(
    model_name_or_path="gpt2",
    torch_dtype="auto",
)


In [6]:

torch_dtype = (
        model_config.torch_dtype
        if model_config.torch_dtype in ["auto", None]
        else getattr(torch, model_config.torch_dtype)
)


In [7]:

model_kwargs = dict(
    trust_remote_code=True,
    torch_dtype=torch_dtype,
    device_map={'':device_string},
)


In [8]:

tokenizer = AutoTokenizer.from_pretrained(model_config.model_name_or_path, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:

## Dataset
dataset_name  = "gsm8k"
train_dataset = load_dataset(dataset_name, name='main', split="train")
eval_dataset  = load_dataset(dataset_name, name='main', split="test")


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [10]:

## Training
trainer = SFTTrainer(
    model=model_config.model_name_or_path,
    model_init_kwargs=model_kwargs,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    max_seq_length=512,
    tokenizer=tokenizer,
    formatting_func=formatting_prompts_func,
)


/home/maquina1/anaconda3/envs/py38_OLD_TRL/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': model_init_kwargs, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/maquina1/anaconda3/envs/py38_OLD_TRL/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:158: UserWarning: You passed `model_init_kwargs` to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/maquina1/anaconda3/envs/py38_OLD_TRL/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:185: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/home/maquina1/anaconda3/envs/py38_OLD_TRL/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

/home/maquina1/anaconda3/envs/py38_OLD_TRL/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [11]:

trainer.train()

  

Step,Training Loss
935,1.747400
1870,1.490900
2805,1.409200


TrainOutput(global_step=2805, training_loss=1.5491774941510694, metrics={'train_runtime': 583.8888, 'train_samples_per_second': 38.396, 'train_steps_per_second': 4.804, 'total_flos': 2834734290048000.0, 'train_loss': 1.5491774941510694, 'epoch': 3.0})

In [12]:

## Uncomment to save model

trainer.save_model(training_args.output_dir)
